In [ ]:
 matrixA = np.array([[2/np.sqrt(5),1/np.sqrt(5)],[1/np.sqrt(5),2/np.sqrt(5)]])
b= np.array([np.sqrt(3)/2,1/2])
qubits = QuantumRegister(4)
Measurement = ClassicalRegister(4)
circuit = QuantumCircuit(qubits, Measurement)
## t = number of values + log(1/(2*error)) 
t = 2.344916
r = 2
last_index = len(qubits)-1
## Loads Data of b in quantum register
circuit.ry(2*np.arccos(matrixA[0][0]), qubits[-1])
print(matrixA[0][0])
##Phase Estimation
 #set superpositon
circuit.h(qubits[1])
circuit.h(qubits[2])
 #troterization
beta = matrixA[0][1]
iterations = 2**(r-1)
gamma = (matrixA[0][0]-matrixA[1][1])/2

for i in (range(1,last_index)):
    for x in range(iterations):
        circuit.crz(((-2*t*gamma*2**(i-1))/r),qubits[last_index-i],qubits[-1])
        circuit.crx(((-2*t*beta*2**(i-1))/r),qubits[last_index-i],qubits[-1])

circuit.swap(qubits[1], qubits[2])
csdg=SdgGate().control(1)
circuit.h(qubits[2])
circuit.append(csdg,[1,2])
circuit.h(qubits[1])
circuit.swap(qubits[1], qubits[2])

#Controlled rotations
circuit.cry(2*np.pi/2**r,qubits[1], qubits[0] )
circuit.cry(np.pi/2**r,qubits[1], qubits[0] )

#inverse QPE
circuit.swap(qubits[1], qubits[2])
circuit.h(qubits[1])
cs=SGate().control(1)
circuit.append(cs,[1,2])
for i in reversed(range(1,last_index)):
    for x in range(iterations):
        circuit.crx(((2*t*beta*2**(i-1))/r),qubits[last_index-i],qubits[-1])
        circuit.crz(((2*t*gamma*2**(i-1))/r),qubits[last_index-i],qubits[-1])
circuit.h(qubits[1])
circuit.h(qubits[2])
print(circuit)

In [ ]:
from qiskit import QuantumCircuit, IBMQ, ClassicalRegister, QuantumRegister
from qiskit.providers.aer import AerSimulator,AerProvider
import numpy as np
from qiskit.circuit.library import phase_estimation
from qiskit.circuit.library.standard_gates import SGate, SdgGate